In [1]:
!pip install sympy numpy
!python navier_stokes_random.py


python3: can't open file '/content/navier_stokes_random.py': [Errno 2] No such file or directory


In [3]:
import sympy as sp
import numpy as np
import csv
import random
import os
from datetime import datetime

# Define symbols
t, x, y, z = sp.symbols('t x y z')
Re = sp.symbols('Re', positive=True)
Re_val = 100  # Example Reynolds number

# Basis functions pool (sin, cos, exp, polynomials)
def basis_funcs(var):
    return [
        sp.sin(sp.pi * var),
        sp.cos(sp.pi * var),
        sp.exp(-var),
        var,
        var**2,
        1
    ]

# Generate a random smooth function as sum of products of basis funcs over variables
def random_func(vars, max_terms=4):
    expr = 0
    for _ in range(random.randint(1, max_terms)):
        coeff = random.uniform(-2, 2)
        term = coeff
        for var in vars:
            f = random.choice(basis_funcs(var))
            term *= f
        expr += term
    return expr

# Prepare grid for numerical evaluation
N = 10
xx = np.linspace(0, 1, N)
yy = np.linspace(0, 1, N)
zz = np.linspace(0, 1, N)
tt = 0  # Fixed time for evaluation

# Create output directory with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f'navier_stokes_output_{timestamp}'
os.makedirs(output_dir, exist_ok=True)

# Number of functions to generate
num_functions = 10000

for func_idx in range(1, num_functions + 1):
    # Generate u, v, w, p expressions
    vars_space = [x, y, z]
    vars_all = [x, y, z, t]

    u_expr = random_func(vars_space)
    v_expr = random_func(vars_space)
    w_expr = random_func(vars_space)
    p_expr = random_func(vars_all, max_terms=3)

    # Compute partial derivatives for residuals
    du_dt = sp.diff(u_expr, t)
    du_dx = sp.diff(u_expr, x)
    du_dy = sp.diff(u_expr, y)
    du_dz = sp.diff(u_expr, z)
    d2u_dx2 = sp.diff(u_expr, x, 2)
    d2u_dy2 = sp.diff(u_expr, y, 2)
    d2u_dz2 = sp.diff(u_expr, z, 2)

    dv_dt = sp.diff(v_expr, t)
    dv_dx = sp.diff(v_expr, x)
    dv_dy = sp.diff(v_expr, y)
    dv_dz = sp.diff(v_expr, z)
    d2v_dx2 = sp.diff(v_expr, x, 2)
    d2v_dy2 = sp.diff(v_expr, y, 2)
    d2v_dz2 = sp.diff(v_expr, z, 2)

    dw_dt = sp.diff(w_expr, t)
    dw_dx = sp.diff(w_expr, x)
    dw_dy = sp.diff(w_expr, y)
    dw_dz = sp.diff(w_expr, z)
    d2w_dx2 = sp.diff(w_expr, x, 2)
    d2w_dy2 = sp.diff(w_expr, y, 2)
    d2w_dz2 = sp.diff(w_expr, z, 2)

    dp_dx = sp.diff(p_expr, x)
    dp_dy = sp.diff(p_expr, y)
    dp_dz = sp.diff(p_expr, z)

    # Define residuals
    res_u = du_dt + u_expr * du_dx + v_expr * du_dy + w_expr * du_dz + dp_dx - (1/Re) * (d2u_dx2 + d2u_dy2 + d2u_dz2)
    res_v = dv_dt + u_expr * dv_dx + v_expr * dv_dy + w_expr * dv_dz + dp_dy - (1/Re) * (d2v_dx2 + d2v_dy2 + d2v_dz2)
    res_w = dw_dt + u_expr * dw_dx + v_expr * dw_dy + w_expr * dw_dz + dp_dz - (1/Re) * (d2w_dx2 + d2w_dy2 + d2w_dz2)

    # Lambdify expressions for numerical evaluation
    try:
        res_u_num = sp.lambdify((x, y, z, t), res_u.subs(Re, Re_val), 'numpy')
        res_v_num = sp.lambdify((x, y, z, t), res_v.subs(Re, Re_val), 'numpy')
        res_w_num = sp.lambdify((x, y, z, t), res_w.subs(Re, Re_val), 'numpy')

        u_num = sp.lambdify((x, y, z), u_expr, 'numpy')
        v_num = sp.lambdify((x, y, z), v_expr, 'numpy')
        w_num = sp.lambdify((x, y, z), w_expr, 'numpy')
        p_num = sp.lambdify((x, y, z, t), p_expr, 'numpy')
    except Exception as e:
        print(f"Error in lambdify for function {func_idx}: {e}")
        continue

    # Collect data rows
    data_rows = []
    residuals = []

    for xi in xx:
        for yi in yy:
            for zi in zz:
                try:
                    u_val = u_num(xi, yi, zi)
                    v_val = v_num(xi, yi, zi)
                    w_val = w_num(xi, yi, zi)
                    p_val = p_num(xi, yi, zi, tt)

                    ru = res_u_num(xi, yi, zi, tt)
                    rv = res_v_num(xi, yi, zi, tt)
                    rw = res_w_num(xi, yi, zi, tt)

                    res_norm = np.sqrt(ru**2 + rv**2 + rw**2)
                    residuals.append(res_norm)

                    data_rows.append([xi, yi, zi, u_val, v_val, w_val, p_val, ru, rv, rw, res_norm])
                except Exception as e:
                    print(f"Error in numerical evaluation at point ({xi}, {yi}, {zi}) for function {func_idx}: {e}")
                    continue

    # Write velocity, pressure, residual data to CSV
    data_filename = os.path.join(output_dir, f'navier_stokes_data_{func_idx}.csv')
    with open(data_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['x', 'y', 'z', 'u', 'v', 'w', 'p', 'res_u', 'res_v', 'res_w', 'res_norm'])
        writer.writerows(data_rows)

    # Write summary stats of residuals
    residuals = np.array(residuals)
    summary = [
        ['Residual Norm', 'Min', 'Max', 'Mean'],
        ['res_norm', np.min(residuals), np.max(residuals), np.mean(residuals)]
    ]

    stats_filename = os.path.join(output_dir, f'navier_stokes_residual_stats_{func_idx}.csv')
    with open(stats_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(summary)

    # Export the generated functions to CSV (string expressions)
    functions_filename = os.path.join(output_dir, f'navier_stokes_functions_{func_idx}.csv')
    with open(functions_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Function', 'Expression'])
        writer.writerow(['u(x,y,z)', str(u_expr)])
        writer.writerow(['v(x,y,z)', str(v_expr)])
        writer.writerow(['w(x,y,z)', str(w_expr)])
        writer.writerow(['p(x,y,z,t)', str(p_expr)])

    print(f"Function {func_idx} processed and data exported.")

print(f"All {num_functions} functions processed. Data saved in '{output_dir}' directory.")


Function 1 processed and data exported.
Function 2 processed and data exported.
Function 3 processed and data exported.
Function 4 processed and data exported.
Function 5 processed and data exported.
Function 6 processed and data exported.
Function 7 processed and data exported.
Function 8 processed and data exported.
Function 9 processed and data exported.
Function 10 processed and data exported.
Function 11 processed and data exported.
Function 12 processed and data exported.
Function 13 processed and data exported.
Function 14 processed and data exported.
Function 15 processed and data exported.
Function 16 processed and data exported.
Function 17 processed and data exported.
Function 18 processed and data exported.
Function 19 processed and data exported.
Function 20 processed and data exported.
Function 21 processed and data exported.
Function 22 processed and data exported.
Function 23 processed and data exported.
Function 24 processed and data exported.
Function 25 processed and

KeyboardInterrupt: 

In [4]:
import sympy as sp
import numpy as np
import csv
import random
import os
from datetime import datetime

# Define symbols
t, x, y, z = sp.symbols('t x y z')
Re = sp.symbols('Re', positive=True)
Re_val = 100  # Example Reynolds number

# Basis functions pool (sin, cos, exp, polynomials)
def basis_funcs(var):
    return [
        sp.sin(sp.pi * var),
        sp.cos(sp.pi * var),
        sp.exp(-var),
        var,
        var**2,
        1
    ]

# Generate a random smooth function as sum of products of basis funcs over variables
def random_func(vars, max_terms=4):
    expr = 0
    for _ in range(random.randint(1, max_terms)):
        coeff = random.uniform(-2, 2)
        term = coeff
        for var in vars:
            f = random.choice(basis_funcs(var))
            term *= f
        expr += term
    return expr

# Prepare grid for numerical evaluation
N = 10
xx = np.linspace(0, 1, N)
yy = np.linspace(0, 1, N)
zz = np.linspace(0, 1, N)
tt = 0  # Fixed time for evaluation

# Create output directory with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f'navier_stokes_output_{timestamp}'
os.makedirs(output_dir, exist_ok=True)

# Number of functions to generate
num_functions = 10000

for func_idx in range(1, num_functions + 1):
    # Generate u, v, w, p expressions
    vars_space = [x, y, z]
    vars_all = [x, y, z, t]

    u_expr = random_func(vars_space)
    v_expr = random_func(vars_space)
    w_expr = random_func(vars_space)
    p_expr = random_func(vars_all, max_terms=3)

    # Compute partial derivatives for residuals
    du_dt = sp.diff(u_expr, t)
    du_dx = sp.diff(u_expr, x)
    du_dy = sp.diff(u_expr, y)
    du_dz = sp.diff(u_expr, z)
    d2u_dx2 = sp.diff(u_expr, x, 2)
    d2u_dy2 = sp.diff(u_expr, y, 2)
    d2u_dz2 = sp.diff(u_expr, z, 2)

    dv_dt = sp.diff(v_expr, t)
    dv_dx = sp.diff(v_expr, x)
    dv_dy = sp.diff(v_expr, y)
    dv_dz = sp.diff(v_expr, z)
    d2v_dx2 = sp.diff(v_expr, x, 2)
    d2v_dy2 = sp.diff(v_expr, y, 2)
    d2v_dz2 = sp.diff(v_expr, z, 2)

    dw_dt = sp.diff(w_expr, t)
    dw_dx = sp.diff(w_expr, x)
    dw_dy = sp.diff(w_expr, y)
    dw_dz = sp.diff(w_expr, z)
    d2w_dx2 = sp.diff(w_expr, x, 2)
    d2w_dy2 = sp.diff(w_expr, y, 2)
    d2w_dz2 = sp.diff(w_expr, z, 2)

    dp_dx = sp.diff(p_expr, x)
    dp_dy = sp.diff(p_expr, y)
    dp_dz = sp.diff(p_expr, z)

    # Define residuals
    res_u = du_dt + u_expr * du_dx + v_expr * du_dy + w_expr * du_dz + dp_dx - (1/Re) * (d2u_dx2 + d2u_dy2 + d2u_dz2)
    res_v = dv_dt + u_expr * dv_dx + v_expr * dv_dy + w_expr * dv_dz + dp_dy - (1/Re) * (d2v_dx2 + d2v_dy2 + d2v_dz2)
    res_w = dw_dt + u_expr * dw_dx + v_expr * dw_dy + w_expr * dw_dz + dp_dz - (1/Re) * (d2w_dx2 + d2w_dy2 + d2w_dz2)

    # Lambdify expressions for numerical evaluation
    try:
        res_u_num = sp.lambdify((x, y, z, t), res_u.subs(Re, Re_val), 'numpy')
        res_v_num = sp.lambdify((x, y, z, t), res_v.subs(Re, Re_val), 'numpy')
        res_w_num = sp.lambdify((x, y, z, t), res_w.subs(Re, Re_val), 'numpy')

        u_num = sp.lambdify((x, y, z), u_expr, 'numpy')
        v_num = sp.lambdify((x, y, z), v_expr, 'numpy')
        w_num = sp.lambdify((x, y, z), w_expr, 'numpy')
        p_num = sp.lambdify((x, y, z, t), p_expr, 'numpy')
    except Exception as e:
        print(f"Error in lambdify for function {func_idx}: {e}")
        continue

    # Collect data rows
    data_rows = []
    residuals = []

    for xi in xx:
        for yi in yy:
            for zi in zz:
                try:
                    u_val = u_num(xi, yi, zi)
                    v_val = v_num(xi, yi, zi)
                    w_val = w_num(xi, yi, zi)
                    p_val = p_num(xi, yi, zi, tt)

                    ru = res_u_num(xi, yi, zi, tt)
                    rv = res_v_num(xi, yi, zi, tt)
                    rw = res_w_num(xi, yi, zi, tt)

                    res_norm = np.sqrt(ru**2 + rv**2 + rw**2)
                    residuals.append(res_norm)

                    data_rows.append([xi, yi, zi, u_val, v_val, w_val, p_val, ru, rv, rw, res_norm])
                except Exception as e:
                    print(f"Error in numerical evaluation at point ({xi}, {yi}, {zi}) for function {func_idx}: {e}")
                    continue

    # Write velocity, pressure, residual data to CSV
    data_filename = os.path.join(output_dir, f'navier_stokes_data_{func_idx}.csv')
    with open(data_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['x', 'y', 'z', 'u', 'v', 'w', 'p', 'res_u', 'res_v', 'res_w', 'res_norm'])
        writer.writerows(data_rows)

    # Write summary stats of residuals
    residuals = np.array(residuals)
    summary = [
        ['Residual Norm', 'Min', 'Max', 'Mean'],
        ['res_norm', np.min(residuals), np.max(residuals), np.mean(residuals)]
    ]

    stats_filename = os.path.join(output_dir, f'navier_stokes_residual_stats_{func_idx}.csv')
    with open(stats_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(summary)

    # Export the generated functions to CSV (string expressions)
    functions_filename = os.path.join(output_dir, f'navier_stokes_functions_{func_idx}.csv')
    with open(functions_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Function', 'Expression'])
        writer.writerow(['u(x,y,z)', str(u_expr)])
        writer.writerow(['v(x,y,z)', str(v_expr)])
        writer.writerow(['w(x,y,z)', str(w_expr)])
        writer.writerow(['p(x,y,z,t)', str(p_expr)])

    print(f"Function {func_idx} processed and data exported.")

print(f"All {num_functions} functions processed. Data saved in '{output_dir}' directory.")


Function 1 processed and data exported.
Function 2 processed and data exported.
Function 3 processed and data exported.
Function 4 processed and data exported.
Function 5 processed and data exported.
Function 6 processed and data exported.
Function 7 processed and data exported.
Function 8 processed and data exported.
Function 9 processed and data exported.
Function 10 processed and data exported.
Function 11 processed and data exported.
Function 12 processed and data exported.
Function 13 processed and data exported.
Function 14 processed and data exported.
Function 15 processed and data exported.
Function 16 processed and data exported.
Function 17 processed and data exported.
Function 18 processed and data exported.
Function 19 processed and data exported.
Function 20 processed and data exported.
Function 21 processed and data exported.
Function 22 processed and data exported.
Function 23 processed and data exported.
Function 24 processed and data exported.
Function 25 processed and

KeyboardInterrupt: 

In [ ]:
import sympy as sp
import numpy as np
import csv
import random
import os
from datetime import datetime

# Initialize symbols
t, x, y, z = sp.symbols('t x y z')
Re = sp.symbols('Re', positive=True)
Re_val = 100  # Example Reynolds number

# Define a comprehensive pool of basis functions
def basis_funcs(var):
    return [
        sp.sin(sp.pi * var),
        sp.cos(sp.pi * var),
        sp.exp(-var),
        sp.exp(var),
        sp.log(var + 1),
        sp.sqrt(var + 1),
        sp.tan(sp.pi * var),
        sp.asin(sp.sin(sp.pi * var)),
        sp.acos(sp.cos(sp.pi * var)),
        sp.atan(sp.tan(sp.pi * var)),
        sp.sinh(var),
        sp.cosh(var),
        sp.tanh(var),
        var,
        var**2,
        var**3,
        1
    ]

# Generate a random smooth function as sum of products of basis functions over variables
def random_func(vars, max_terms=4):
    expr = 0
    for _ in range(random.randint(1, max_terms)):
        coeff = random.uniform(-2, 2)
        term = coeff
        for var in vars:
            f = random.choice(basis_funcs(var))
            term *= f
        expr += term
    return expr

# Prepare grid for numerical evaluation
N = 10
xx = np.linspace(0, 1, N)
yy = np.linspace(0, 1, N)
zz = np.linspace(0, 1, N)
tt = 0  # Fixed time for evaluation

# Create output directory with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f'navier_stokes_output_{timestamp}'
os.makedirs(output_dir, exist_ok=True)

# Number of functions to generate
num_functions = 10000

for func_idx in range(1, num_functions + 1):
    # Generate u, v, w, p expressions
    vars_space = [x, y, z]
    vars_all = [x, y, z, t]

    u_expr = random_func(vars_space)
    v_expr = random_func(vars_space)
    w_expr = random_func(vars_space)
    p_expr = random_func(vars_all, max_terms=3)

    # Compute partial derivatives for residuals
    du_dt = sp.diff(u_expr, t)
    du_dx = sp.diff(u_expr, x)
    du_dy = sp.diff(u_expr, y)
    du_dz = sp.diff(u_expr, z)
    d2u_dx2 = sp.diff(u_expr, x, 2)
    d2u_dy2 = sp.diff(u_expr, y, 2)
    d2u_dz2 = sp.diff(u_expr, z, 2)

    dv_dt = sp.diff(v_expr, t)
    dv_dx = sp.diff(v_expr, x)
    dv_dy = sp.diff(v_expr, y)
    dv_dz = sp.diff(v_expr, z)
    d2v_dx2 = sp.diff(v_expr, x, 2)
    d2v_dy2 = sp.diff(v_expr, y, 2)
    d2v_dz2 = sp.diff(v_expr, z, 2)

    dw_dt = sp.diff(w_expr, t)
    dw_dx = sp.diff(w_expr, x)
    dw_dy = sp.diff(w_expr, y)
    dw_dz = sp.diff(w_expr, z)
    d2w_dx2 = sp.diff(w_expr, x, 2)
    d2w_dy2 = sp.diff(w_expr, y, 2)
    d2w_dz2 = sp.diff(w_expr, z, 2)

    dp_dx = sp.diff(p_expr, x)
    dp_dy = sp.diff(p_expr, y)
    dp_dz = sp.diff(p_expr, z)

    # Define residuals
    res_u = du_dt + u_expr * du_dx + v_expr * du_dy + w_expr * du_dz + dp_dx - (1/Re) * (d2u_dx2 + d2u_dy2 + d2u_dz2)
    res_v = dv_dt + u_expr * dv_dx + v_expr * dv_dy + w_expr * dv_dz + dp_dy - (1/Re) * (d2v_dx2 + d2v_dy2 + d2v_dz2)
    res_w = dw_dt + u_expr * dw_dx + v_expr * dw_dy + w_expr * dw_dz + dp_dz - (1/Re) * (d2w_dx2 + d2w_dy2 + d2w_dz2)

    # Lambdify expressions for numerical evaluation
    try:
        res_u_num = sp.lambdify((x, y, z, t), res_u.subs(Re, Re_val), 'numpy')
        res_v_num = sp.lambdify((x, y, z, t), res_v.subs(Re, Re_val), 'numpy')
        res_w_num = sp.lambdify((x, y, z, t), res_w.subs(Re, Re_val), 'numpy')

        u_num = sp.lambdify((x, y, z), u_expr, 'numpy')
        v_num = sp.lambdify((x, y, z), v_expr, 'numpy')
        w_num = sp.lambdify((x, y, z), w_expr, 'numpy')
        p_num = sp.lambdify((x, y, z, t), p_expr, 'numpy')
    except Exception as e:
        print(f"Error in lambdify for function {func_idx}: {e}")
        continue

    # Collect data rows
    data_rows = []
    residuals = []

    for xi in xx:
        for yi in yy:
            for zi in zz:
                try:
                    u_val = u_num(xi, yi, zi)
                    v_val = v_num(xi, yi, zi)
                    w_val = w_num(xi, yi, zi)
                    p_val = p_num(xi, yi, zi, tt)

                    ru = res_u_num(xi, yi, zi, tt)
                    rv = res_v_num(xi, yi, zi, tt)
                    rw = res_w_num(xi, yi, zi, tt)

                    res_norm = np.sqrt(ru**2 + rv**2 + rw**2)
                    residuals.append(res_norm)

                    data_rows.append([xi, yi, zi, u_val, v_val, w_val, p_val, ru, rv, rw, res_norm])
                except Exception as e:
                    print(f"Error in numerical evaluation at point ({xi}, {yi}, {zi}) for function {func_idx}: {e}")
                    continue

    # Write velocity, pressure, residual data to CSV
    data_filename = os.path.join(output_dir, f'navier_stokes_data_{func_idx}.csv')
    with open(data_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['x', 'y', 'z', 'u', 'v', 'w', 'p', 'res_u', 'res_v', 'res_w', 'res_norm'])
        writer.writerows(data_rows)

    # Write summary stats of residuals
    residuals = np.array(residuals)
    summary = [
        ['Residual Norm', 'Min', 'Max', 'Mean'],
        ['res_norm', np.min(residuals), np.max(residuals), np.mean(residuals)]
    ]

    stats_filename = os.path.join(output_dir, f'navier_stokes_residual_stats_{func_idx}.csv')
    with open(stats_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(summary)

    # Export the generated functions to CSV (string expressions)
    functions_filename = os.path.join(output_dir, f'navier_stokes_functions_{func_idx}.csv')
    with open(functions_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Function', 'Expression'])
        writer.writerow(['u(x,y,z)', str(u_expr)])
        writer.writerow(['v(x,y,z)', str(v_expr)])
        writer.writerow(['w(x,y,z)', str(w_expr)])
        writer.writerow(['p(x,y,z,t)', str(p_expr)])

    print(f"Function {func_idx} processed and data exported.")

print(f"All {num_functions} functions processed. Data saved in '{output_dir}' directory.")


<lambdifygenerated-9082>:2: RuntimeWarning: invalid value encountered in scalar divide
  return -0.0094977219370479*pi**2*z**3*sin(pi*y) + 0.0569863316222874*z*sin(pi*y) - 1.13663116888587*sqrt(t + 1)*tan(pi*y)*cosh(z)*arctan(tan(pi*x)) + (-1.04054113974045*y**3*z**3*sin(pi*x) - 0.214445197523836*z*tan(pi*x)*arctan(tan(pi*y)))*(0.0945907220015121*pi*exp(-y)*cos(pi*x)*arccos(cos(pi*z)) - 0.411785785517942*exp(-x)*tan(pi*y)*arccos(cos(pi*z)) + 0.924422755572211*exp(-x)*arcsin(sin(pi*z))) - 0.00823571571035885*pi**2*(tan(pi*y)**2 + 1)*exp(-x)*tan(pi*y)*arccos(cos(pi*z)) - 0.917161010159234*(-0.94977219370479*pi*z**3*cos(pi*y) + 0.411785785517942*pi*(tan(pi*y)**2 + 1)*exp(-x)*arccos(cos(pi*z)) - 0.0945907220015121*exp(-y)*sin(pi*x)*arccos(cos(pi*z)))*sinh(x)*arcsin(sin(pi*z))*arctan(tan(pi*y)) + (-2.84931658111437*z**2*sin(pi*y) + 0.0945907220015121*pi*exp(-y)*sin(pi*x)*sin(pi*z)/sqrt(1 - cos(pi*z)**2) + 0.411785785517942*pi*exp(-x)*sin(pi*z)*tan(pi*y)/sqrt(1 - cos(pi*z)**2) - 0.9244227555

Function 1 processed and data exported.


<lambdifygenerated-9087>:2: RuntimeWarning: invalid value encountered in scalar divide
  return 0.0459147400566597*y**3*exp(-x)*sin(pi*t)*arctan(tan(pi*z)) - 0.000849282940525899*pi**2*z*(tan(pi*x)**2 + 1)*tan(pi*x)*sinh(y) - 0.00042464147026295*z*tan(pi*x)*sinh(y) + (-1.36489975092434*sinh(z)*arccos(cos(pi*x))*arctan(tan(pi*y)) - 0.848689573803171*arccos(cos(pi*z))*arcsin(sin(pi*y)))*(0.042464147026295*z*tan(pi*x)*cosh(y) + 1.91139065785961*(1 - tanh(y)**2)*exp(-x)*sin(pi*z) + 0.839900558409625*pi*cos(pi*z)*arccos(cos(pi*x)) - 0.856380553324392*sinh(y)*tanh(z)) - 0.0382278131571922*(tanh(y)**2 - 1)*exp(-x)*sin(pi*z)*tanh(y) + (0.609367100194387*z**3*tanh(x)*arcsin(sin(pi*y)) + 1.66867526209114*sqrt(y + 1)*sinh(z) + 0.393361730238579*exp(x)*tanh(z))*(-0.856380553324392*(1 - tanh(z)**2)*cosh(y) - 0.839900558409625*pi*sin(pi*z)*arccos(cos(pi*x))*arctan(tan(pi*y)) + 0.042464147026295*tan(pi*x)*sinh(y) + 1.91139065785961*pi*exp(-x)*cos(pi*z)*tanh(y)) + (1/100)*(1.71276110664878*(tanh(z)**2

Function 2 processed and data exported.


<lambdifygenerated-9095>:2: RuntimeWarning: divide by zero encountered in scalar divide
  return -1.98715518763631*pi*x**2*sin(pi*y)*sinh(t)*arctan(tan(pi*z)) + 0.00489951386368254*y*(tanh(z)**2 - 1)*tanh(z)*arcsin(sin(pi*x)) - 0.00244975693184127*pi**2*y*sin(pi*x)*tanh(z)/sqrt(1 - sin(pi*x)**2) + 0.00244975693184127*pi**2*y*sin(pi*x)*cos(pi*x)**2*tanh(z)/(1 - sin(pi*x)**2)**(3/2) - 1/100*z**3*(-0.403274058069338*log(x + 1)/(y + 1)**(3/2) - 1.64574875590861*pi**2*cos(pi*y)/sqrt(1 - cos(pi*y)**2) + 1.64574875590861*pi**2*sin(pi*y)**2*cos(pi*y)/(1 - cos(pi*y)**2)**(3/2)) + 0.0161309623227735*z**3*sqrt(y + 1)/(x + 1)**2 - 0.0967857739366412*z*sqrt(y + 1)*log(x + 1) + 0.0987449253545167*z*arccos(cos(pi*y)) + (0.73858131665777*exp(x)*cos(pi*z)*cosh(y) - 0.761215958671245*exp(y)*log(x + 1)*arctan(tan(pi*z)))*(-0.244975693184127*y*(1 - tanh(z)**2)*arcsin(sin(pi*x)) + 4.83928869683206*z**2*sqrt(y + 1)*log(x + 1) - 4.93724626772584*z**2*arccos(cos(pi*y))) + (-0.244975693184127*pi*y*cos(pi*x)*ta

...

<lambdifygenerated-9500>:2: RuntimeWarning: invalid value encountered in scalar divide
  return 2.18300802770398*t**3*x*sqrt(z + 1)*arcsin(sin(pi*y)) - 1.72978861149661*t**2*(1 - tanh(x)**2)*sinh(z)*arctan(tan(pi*y)) + 1.84934267947009*x**3*(y + 1)*arccos(cos(pi*z))**2 - 0.00961598325567929*pi**2*x**2*(1 + sin(pi*z)**2/(cos(pi*z)**2 - 1))*sqrt(y + 1)*cos(pi*z)/sqrt(1 - cos(pi*z)**2) + 0.480799162783964*x**2*(-1.39650311761146*sqrt(y + 1)*exp(z)*tan(pi*x) - 1.54037205149424*exp(-z)*cos(pi*y) - 0.792224879285898*exp(-y)*tan(pi*z)*cosh(x))*arccos(cos(pi*z))/sqrt(y + 1) + 0.00240399581391982*x**2*arccos(cos(pi*z))/(y + 1)**(3/2) - 0.350626171577517*pi*x**2*sqrt(y + 1)*exp(x)*exp(z)*sin(pi*z)*arcsin(sin(pi*y))/sqrt(1 - cos(pi*z)**2) - 0.280138805677684*pi*z*sqrt(t + 1)*(tan(pi*x)**2 + 1)*exp(-y) - 0.0192319665113586*sqrt(y + 1)*arccos(cos(pi*z))
<lambdifygenerated-9500>:2: RuntimeWarning: divide by zero encountered in scalar divide
  return 2.18300802770398*t**3*x*sqrt(z + 1)*arcsin(sin(pi*

Function 61 processed and data exported.


<lambdifygenerated-9507>:2: RuntimeWarning: invalid value encountered in scalar divide
  return -0.0187570209128315*pi**2*z**2*(-1 + cos(pi*y)**2/(1 - sin(pi*y)**2))*exp(-x)*sin(pi*y)/sqrt(1 - sin(pi*y)**2) - 0.0736393160437139*pi*z**2*exp(-y)*arccos(cos(pi*t)) - 0.0187570209128315*z**2*exp(-x)*arcsin(sin(pi*y)) + (3.75140418256631*z*exp(-x)*arcsin(sin(pi*y)) + 1.6883742923575*pi*sin(pi*z)*arccos(cos(pi*x))*arctan(tan(pi*y)))*(-1.15813297268689*x**2*z*arcsin(sin(pi*y)) - 1.88466852510348*z**2*sin(pi*y) - 0.997958751143821*exp(x)*exp(y)) + (-1.87570209128315*z**2*exp(-x)*arcsin(sin(pi*y)) - 1.6883742923575*pi*sin(pi*x)*cos(pi*z)*arctan(tan(pi*y))/sqrt(1 - cos(pi*x)**2))*(1.87570209128315*z**2*exp(-x)*arcsin(sin(pi*y)) - 1.6883742923575*cos(pi*z)*arccos(cos(pi*x))*arctan(tan(pi*y))) + (1.87570209128315*pi*z**2*exp(-x)*cos(pi*y)/sqrt(1 - sin(pi*y)**2) - 1.6883742923575*pi*cos(pi*z)*arccos(cos(pi*x)))*(-1.27715626519167*x**3*sin(pi*z)*arccos(cos(pi*y)) - 0.741498787116064*y**3*exp(z)*tan(p

Function 62 processed and data exported.


<lambdifygenerated-9515>:2: RuntimeWarning: invalid value encountered in scalar divide
  return -0.733475473382465*pi*z*cosh(x)*tanh(t) - 1.53225119731146*pi*(-0.612770056010606*z**3*exp(-x)*arccos(cos(pi*y)) - 0.936883450678246*sqrt(x + 1)*tan(pi*y)*arcsin(sin(pi*z)) - 1.11662756109333*sqrt(z + 1)*tanh(y)*arcsin(sin(pi*x)))*exp(x)*sin(pi*y)*cos(pi*z) - 1.53225119731146*(1.1580275118489*exp(x)*cos(pi*z)*cosh(y) + 0.937960762614036*exp(z)*cosh(y)*arcsin(sin(pi*x)) + 1.48220122647158*sinh(x)*sinh(z)*arcsin(sin(pi*y)) + 1.87831998659598*exp(-z)*tan(pi*x)*cosh(y))*exp(x)*sin(pi*y)*sin(pi*z) + 2.34779373166241*pi*exp(2*x)*sin(pi*y)*sin(pi*z)**2*cos(pi*y) - 0.0306450239462293*pi**2*exp(x)*sin(pi*y)*sin(pi*z) + 0.0153225119731146*exp(x)*sin(pi*y)*sin(pi*z) - 0.268617303508982*exp(-y)*exp(z)*cosh(t)*cosh(x) + 0.326733613985851*pi*exp(-t)*exp(-z)*sin(pi*y)*cos(pi*x)/sqrt(1 - cos(pi*y)**2)
<lambdifygenerated-9516>:2: RuntimeWarning: invalid value encountered in scalar divide
  return 0.006127700